# Is it possible to use full kinetic model for non-linear fit of competitive inhibition?

#### Import reqired python modules

In [483]:
import numpy as np # numeric python module for vectorized operations on arrays 
import pandas # convenience wrap for numpy 
from scipy.integrate import odeint # function for solving ordinary differential equations
from scipy.special import lambertw # implementation of Lambert W function
from scipy.optimize import minimize,basinhopping,differential_evolution #optimization algorithms
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt # plotting graphics library
%matplotlib auto
plt.ioff() # I prefer to output graphs into separate windows rather than showing them inline.
#%matplotlib inline 

Using matplotlib backend: Qt5Agg


# Set up main variables

In [484]:
#linear scale of [S]o
listSo=  [0,20,40,60,80,100,120,140,160] # initial substrate  concentrations (list if floats)
Smax=160
#reciprocal scale of [S]o
#listSo=  [Smax/7,Smax/6,Smax/5,Smax/4,Smax/3,Smax/2,Smax/1] # initial substrate  concentrations (list if floats)
listS_STR = list(map(str, listSTR))
listI=  [0,20,40,60,80,100,120,140,160]
listI_STR = list(map(str, listI))

noise_level=.01 # we may add some random noise to the data
noise=False #if False, no noise will be added to original data 
time_cutoff=10 # number of data points used for linear regression to estimate [S]o 

simulate_data=True #if False, data will be loaded from '../data/simulatedMM_variable_S.csv'  # Input experimental data
simulate_rates=True #if False, data will be loaded from '../data/rates.csv'


#### Generate a data set for competitive inhibition model;
$$EI \underset{k_{3}}
{\overset{k_{-3}}{\rightleftharpoons}} E+S+I \underset{k_{-1}}
{\overset{k_{1}}{\rightleftharpoons}} ES\overset{k_{2}}{\rightarrow}E+P$$
$$\left [ \frac{\mathrm{d} [S]}{\mathrm{d} t}=k_{-1}[ES]-k_{1}[E][S];\; \frac{\mathrm{d} [E]}{\mathrm{d} t}=k_{-1}[ES]+k_{2}[ES]-k_{1}[E][S]+k_{-3}[EI]-k_{3}[E][I];\; \frac{\mathrm{d} [ES]}{\mathrm{d} t}=k_{1}[E][S]]-k_{-1}[ES]-k_{2}[ES];\;\frac{\mathrm{d} [EI]}{\mathrm{d} t}=k_{3}[E][I]]-k_{-3}[EI];\;\frac{\mathrm{d} [I]}{\mathrm{d} t}=-k_{3}[E][I]]+k_{-3}[EI];\;\frac{\mathrm{d} [P]}{\mathrm{d} t}=k_{2}[ES] \right ]$$

In [485]:
def model_full(init,t,*args): 
    '''Complete Sequential kinetics model'''
    S,E,ES,I,EI,P=init
    k1,k1r,k2,k3,k3r=args
    dSdt=-k1*S*E+k1r*ES
    dEdt=-k1*S*E+k1r*ES+k2*ES-k3*E*I+k3r*EI
    
    dESdt=k1*S*E-k1r*ES-k2*ES
    
    dIdt=-k3*E*I+k3r*EI
    dEIdt=k3*E*I-k3r*EI
    
    dPdt=k2*ES
    return (dSdt,dIdt,dEIdt,dEdt,dESdt,dPdt)
So=160
E=0.01
I=0.01
ES=0
EI=0
P=0
k1=10
k1r=1
k3=0.1
k3r=0.1
k2=400
KM=(k2+k1r)/k1
Vm=k2*E
kcat=k2
exact='True MM parameters: KM={}; Vmax={}; Vmax/KM={}'.format(KM,Vm,Vm/KM)
t = np.linspace(0,99,100) # make an array of time points
init=(So,E,ES,I,EI,P)# pack initial conditions into tuple
args=(k1,k1r,k2,k3,k3r) # pack arguments into tuple
z = odeint(model_full,init,t,args) # solve model_full by integration of ordinary differential equations (ODE)
z



array([[ 1.60000000e+002,  1.00000000e-002,  0.00000000e+000,
         1.00000000e-002,  0.00000000e+000,  0.00000000e+000],
       [-9.99999694e-002,  3.27980652e+004, -3.27980552e+004,
        -3.99845071e+003,  3.67965159e+004, -3.66364159e+004],
       [ 1.69523417e-315,  1.70721380e-315,  1.40540973e-315,
         1.55627256e-315,  2.01607242e-315,  1.68539167e-315],
       [ 8.90663918e-316,  2.14730123e-315,  1.38968255e-315,
         5.99705102e-316,  2.12802334e-315,  2.07080461e-315],
       [ 2.00123938e-315,  1.69525346e-315,  1.69890986e-315,
         6.12872465e-316,  1.71182980e-315,  1.14955408e-315],
       [ 2.14733633e-315,  2.07078659e-315,  1.27194821e-315,
         2.14727530e-315,  1.71182474e-315,  2.14733127e-315],
       [ 1.55625801e-315,  2.12806034e-315,  1.55625295e-315,
         2.01608348e-315,  2.01609139e-315,  2.07082074e-315],
       [ 1.55627856e-315,  2.14731103e-315,  1.69526959e-315,
         1.70720329e-315,  1.56010453e-315,  1.70675871e-315],


# Full kinetic model fails presumably due to numeric instability